# Python &rarr; OKL Sandbox

AST Grammar info can be found in the [ast docs](https://docs.python.org/3/library/ast.html#abstract-grammar)

In [1]:
%load_ext autoreload
%autoreload 2

In [14]:
from typing import List
import ast
import inspect

from occa import okl
from occa.okl import py2okl
from occa.okl.types import Const, Exclusive, Shared

In [26]:
def add(a: Const[float], 
        b: Const[float]) -> float:
    return a + b

@okl.kernel
def add_vectors(a: Const[List[float]],
                b: Const[List[float]],
                ab: List[float]) -> None:
    for i in okl.range(len(a)).tile(16):
        foo: bool = True
        s_foo: Shared[List[float, 20, 30]]
        e_foo: Exclusive[float]
        bar: Const[List[float, 2]] = [1,2]
        ab[i] = add(a[i], b[i])

In [27]:
print(add_vectors.__okl_source__)

float add(const float a,
          const float b);

float add(const float a,
          const float b) {
  return a + b;
}

@kernel void add_vectors(const float *a,
                         const float *b,
                         float *ab) {
  for (int i = 0; i < 10; ++i) {
    bool foo = true;
    @shared float s_foo[20][30];
    @exclusive float e_foo;
    const float bar[2] = {1, 2};
    ab[i] = add(a[i], b[i]);
  }
}


In [107]:
foo = py2okl.Oklifier.get_last_error_node()

In [108]:
foo.elts

[<_ast.Num at 0x10e051668>, <_ast.Num at 0x10e0518d0>]

In [59]:
foo.value

In [61]:
py2okl.py2okl(foo.annotation)

'float'

In [167]:
py2okl.py2okl(iter)

'okl.range(len(a)).tile(foo)'

In [132]:
py2okl.py2okl(iter.args)

['16']

In [136]:
iter.func.attr

'tile'

In [137]:
py2okl.py2okl(iter.func.value)

'okl.range(len(a))'

In [138]:
py2okl.py2okl(iter.func.value.args)

['len(a)']

In [141]:
iter.func.value.func.attr

'range'

In [143]:
py2okl.py2okl(iter.func.value.func.value)

'okl'

In [144]:
py2okl.py2okl(iter.func.value.func)

'okl.range'

In [145]:
py2okl.py2okl(iter.func)

'okl.range(len(a)).tile'